<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/retrievers/auto_vs_recursive_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 比较结构化检索方法（自动检索 vs. 递归检索）

在一个简单的RAG系统中，输入文档集会被分块、嵌入，并存储到一个向量数据库集合中。检索只需通过嵌入相似度获取前k个文档。

如果文档集很大，这种方法可能会失败 - 很难消除原始块的歧义，并且无法保证筛选出包含相关上下文的文档集。

在本指南中，我们探讨**结构化检索** - 更高级的查询算法，利用文档内部结构实现更精准的检索。我们比较以下两种方法：

- **元数据过滤器 + 自动检索**：为每个文档打上正确的元数据标签。在查询时，使用自动检索来推断元数据过滤器，并通过查询字符串进行语义搜索。
- **存储文档层次结构（摘要 -> 原始块）+ 递归检索**：嵌入文档摘要，并将其映射到每个文档的原始块集合。在查询时，进行递归检索，先获取摘要，然后再获取文档。


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-vector-stores-weaviate

In [ ]:
!pip install llama-index

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import logging
import sys
from llama_index.core import SimpleDirectoryReader
from llama_index.core import SummaryIndex

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
wiki_titles = ["Michael Jordan", "Elon Musk", "Richard Branson", "Rihanna"]
wiki_metadatas = {
    "Michael Jordan": {
        "category": "Sports",
        "country": "United States",
    },
    "Elon Musk": {
        "category": "Business",
        "country": "United States",
    },
    "Richard Branson": {
        "category": "Business",
        "country": "UK",
    },
    "Rihanna": {
        "category": "Music",
        "country": "Barbados",
    },
}

In [ ]:
from pathlib import Pathimport requestsfor title in wiki_titles:    response = requests.get(        "https://en.wikipedia.org/w/api.php",        params={            "action": "query",            "format": "json",            "titles": title,            "prop": "extracts",            # 'exintro': True,            "explaintext": True,        },    ).json()    page = next(iter(response["query"]["pages"].values()))    wiki_text = page["extract"]    data_path = Path("data")    if not data_path.exists():        Path.mkdir(data_path)    with open(data_path / f"{title}.txt", "w") as fp:        fp.write(wiki_text)

In [ ]:
# 加载所有维基文档docs_dict = {}for wiki_title in wiki_titles:    doc = SimpleDirectoryReader(        input_files=[f"data/{wiki_title}.txt"]    ).load_data()[0]    doc.metadata.update(wiki_metadatas[wiki_title])    docs_dict[wiki_title] = doc

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import LlamaDebugHandler, CallbackManager
from llama_index.core.node_parser import SentenceSplitter


llm = OpenAI("gpt-4")
callback_manager = CallbackManager([LlamaDebugHandler()])
splitter = SentenceSplitter(chunk_size=256)

## 元数据过滤器 + 自动检索

在这种方法中，我们使用元数据（类别、国家）为每个文档打上标记，并将其存储在 Weaviate 向量数据库中。

在检索时，我们会执行“自动检索”来推断相关的元数据过滤器集合。


In [ ]:
## 设置 Weaviateimport weaviate# 云端auth_config = weaviate.AuthApiKey(api_key="<api_key>")client = weaviate.Client(    "https://llama-index-test-v0oggsoz.weaviate.network",    auth_client_secret=auth_config,)

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

In [ ]:
# 从集合中首先删除项目client.schema.delete_class("LlamaIndex")

In [ ]:
from llama_index.core import StorageContext# 如果您想以后加载索引，请确保给它一个名称！vector_store = WeaviateVectorStore(    weaviate_client=client, index_name="LlamaIndex")storage_context = StorageContext.from_defaults(vector_store=vector_store)# 注意：您也可以选择手动定义index_name。# index_name = "test_prefix"# vector_store = WeaviateVectorStore(weaviate_client=client, index_name=index_name)

In [ ]:
# 验证模式是否已创建class_schema = client.schema.get("LlamaIndex")display(class_schema)

{'class': 'LlamaIndex',
 'description': 'Class for LlamaIndex',
 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
  'cleanupIntervalSeconds': 60,
  'stopwords': {'additions': None, 'preset': 'en', 'removals': None}},
 'multiTenancyConfig': {'enabled': False},
 'properties': [{'dataType': ['text'],
   'description': 'Text property',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'text',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'The ref_doc_id of the Node',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'ref_doc_id',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'node_info (in JSON)',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'node_info',
   'tokenization': 'word'},
  {'dataType': ['text'],
   'description': 'The relationships of the node (in JSON)',
   'indexFilterable': True,
   'indexSearchable': True,
   'name': 'relationships',
   'tokenization': 'word'}],
 

In [ ]:
index = VectorStoreIndex(    [],    storage_context=storage_context,    transformations=[splitter],    callback_manager=callback_manager,)# 将文档添加到索引for wiki_title in wiki_titles:    index.insert(docs_dict[wiki_title])

**********
Trace: index_construction
**********
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: insert
**********
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.opena

In [ ]:
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo


vector_store_info = VectorStoreInfo(
    content_info="brief biography of celebrities",
    metadata_info=[
        MetadataInfo(
            name="category",
            type="str",
            description=(
                "Category of the celebrity, one of [Sports, Entertainment,"
                " Business, Music]"
            ),
        ),
        MetadataInfo(
            name="country",
            type="str",
            description=(
                "Country of the celebrity, one of [United States, Barbados,"
                " Portugal]"
            ),
        ),
    ],
)
retriever = VectorIndexAutoRetriever(
    index,
    vector_store_info=vector_store_info,
    llm=llm,
    callback_manager=callback_manager,
    max_top_k=10000,
)

In [ ]:
# 注意：将“将top-k设置为10000”是一种技巧，用于返回所有数据。# 现在自动检索将始终返回固定的top-k，有一个TODO可以允许它为None# 以获取所有数据。# 因此理论上可能让LLM推断出一个None的top-k值。nodes = retriever.retrieve(    "告诉我关于来自美国的名人，将top k设置为10000")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: Tell me about a celebrity
Using query str: Tell me about a celebrity
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('country', '==', 'United States')]
Using filters: [('country', '==', 'United States')]
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_retrieve ->  4.232108 seconds
**********


In [ ]:
print(f"Number of nodes: {len(nodes)}")
for node in nodes[:10]:
    print(node.node.get_content())

Number of nodes: 2
In December 2023, Judge Laurel Beeler ruled that Musk must testify again for the SEC.  


== Public perception ==

Though his ventures were influential within their own industries in the 2000s, Musk only became a public figure in the early 2010s. He has been described as an eccentric who makes spontaneous and controversial statements, contrary to other billionaires who prefer reclusiveness to protect their businesses. Vance described people's opinions of Musk as polarized due to his "part philosopher, part troll" role on Twitter.Musk was a partial inspiration for the characterization of Tony Stark in the Marvel film Iron Man (2008). Musk also had a cameo appearance in the film's 2010 sequel, Iron Man 2. Musk has made cameos and appearances in other films such as Machete Kills (2013), Why Him? (2016), and Men in Black: International (2019). Television series in which he has appeared include The Simpsons ("The Musk Who Fell to Earth", 2015), The Big Bang Theory ("The P

In [ ]:
nodes = retriever.retrieve(
    "Tell me about the childhood of a popular sports celebrity in the United"
    " States"
)
for node in nodes:
    print(node.node.get_content())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: childhood of a popular sports celebrity
Using query str: childhood of a popular sports celebrity
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('category', '==', 'Sports'), ('country', '==', 'United States')]
Using filters: [('category', '==', 'Sports'), ('country', '==', 'United States')]
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_retrieve ->  3.546065 seconds
**********
It was announced on November 30, 2013, that the two we

In [ ]:
nodes = retriever.retrieve(
    "Tell me about the college life of a billionaire who started at company at"
    " the age of 16"
)
for node in nodes:
    print(node.node.get_content())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: college life of a billionaire who started at company at the age of 16
Using query str: college life of a billionaire who started at company at the age of 16
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: []
Using filters: []
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_retrieve ->  2.60008 seconds
**********
After his parents divorced in 1980, Musk chose to live primarily with his father. Musk later regretted his decision and bec

In [ ]:
nodes = retriever.retrieve("Tell me about the childhood of a UK billionaire")
for node in nodes:
    print(node.node.get_content())

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using query str: childhood of a UK billionaire
Using query str: childhood of a UK billionaire
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using filters: [('category', '==', 'Business'), ('country', '==', 'United Kingdom')]
Using filters: [('category', '==', 'Business'), ('country', '==', 'United Kingdom')]
INFO:llama_index.indices.vector_store.retrievers.auto_retriever.auto_retriever:Using top_k: 2
Using top_k: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: query
    |_retrieve ->  3.565899 seconds
**********


## 构建递归检索器以获取文档摘要

在这个任务中，我们将构建一个递归检索器，用于从文档摘要中检索信息。


In [ ]:
from llama_index.core.schema import IndexNode

In [ ]:
# 定义顶层节点和向量检索器nodes = []vector_query_engines = {}vector_retrievers = {}for wiki_title in wiki_titles:    # 构建向量索引    vector_index = VectorStoreIndex.from_documents(        [docs_dict[wiki_title]],        transformations=[splitter],        callback_manager=callback_manager,    )    # 定义查询引擎    vector_query_engine = vector_index.as_query_engine(llm=llm)    vector_query_engines[wiki_title] = vector_query_engine    vector_retrievers[wiki_title] = vector_index.as_retriever()    # 保存摘要    out_path = Path("summaries") / f"{wiki_title}.txt"    if not out_path.exists():        # 使用LLM生成的摘要        summary_index = SummaryIndex.from_documents(            [docs_dict[wiki_title]], callback_manager=callback_manager        )        summarizer = summary_index.as_query_engine(            response_mode="tree_summarize", llm=llm        )        response = await summarizer.aquery(            f"给我一个关于{wiki_title}的摘要"        )        wiki_summary = response.response        Path("summaries").mkdir(exist_ok=True)        with open(out_path, "w") as fp:            fp.write(wiki_summary)    else:        with open(out_path, "r") as fp:            wiki_summary = fp.read()    print(f"**{wiki_title}的摘要：{wiki_summary}")    node = IndexNode(text=wiki_summary, index_id=wiki_title)    nodes.append(node)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: index_construction
**********
**Summary for Michael Jordan: Michael Jordan, often referred to as MJ, is a retired professional basketball player from the United States who is widely considered one of the greatest players in the history of the sport. He played 15 seasons in the NBA, primarily with the Chicago Bulls, and won six NBA championships. His individual accolades include six NBA Finals MVP awards, ten NBA scoring titles, five NBA MVP awards, and fourteen NBA All-Star Game selections. He also h

In [ ]:
# 定义顶层检索器top_vector_index = VectorStoreIndex(    nodes, transformations=[splitter], callback_manager=callback_manager)top_vector_retriever = top_vector_index.as_retriever(similarity_top_k=1)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
**********
Trace: index_construction
**********


In [ ]:
# 定义递归检索器from llama_index.core.retrievers import RecursiveRetrieverfrom llama_index.core.query_engine import RetrieverQueryEnginefrom llama_index.core import get_response_synthesizer

In [ ]:
# 注意：可以将`agents`字典作为`query_engine_dict`传递，因为每个代理都可以用作查询引擎recursive_retriever = RecursiveRetriever(    "vector",    retriever_dict={"vector": top_vector_retriever, **vector_retrievers},    # query_engine_dict=vector_query_engines,    verbose=True,)

In [ ]:
# 运行递归检索器nodes = recursive_retriever.retrieve(    "告诉我一个来自美国的名人")for node in nodes:    print(node.node.get_content())

Retrieving with query id None: Tell me about a celebrity from the United States
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieved node with id, entering: Michael Jordan
Retrieving with query id Michael Jordan: Tell me about a celebrity from the United States
Retrieving text node: In 1999, an ESPN survey of journalists, athletes and other sports figures ranked Jordan the greatest North American athlete of the 20th century. Jordan placed second to Babe Ruth in the Associated Press' December 1999 list of 20th century athletes. In addition, the Associated Press voted him the greatest basketball player of the 20th century. Jordan has also appeared on the front cover of Sports Illustrated a record 50 times. In the September 1996 issue of Sport, which was the publication's 50th-anniversary issue, Jordan was named the greatest athlete of the past 50 years.Jordan's athletic lea

In [ ]:
nodes = recursive_retriever.retrieve(
    "Tell me about the childhood of a billionaire who started at company at"
    " the age of 16"
)
for node in nodes:
    print(node.node.get_content())

Retrieving with query id None: Tell me about the childhood of a billionaire who started at company at the age of 16
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Retrieved node with id, entering: Richard Branson
Retrieving with query id Richard Branson: Tell me about the childhood of a billionaire who started at company at the age of 16
Retrieving text node: He attended Stowe School, a private school in Buckinghamshire until the age of sixteen.Branson has dyslexia, and had poor academic performance; on his last day at school, his headmaster, Robert Drayson, told him he would either end up in prison or become a millionaire. 
Branson has also talked openly about having ADHD.
Branson's parents were supportive of his endeavours from an early age. His mother was an entrepreneur; one of her most successful ventures was building and selling wooden tissue boxes and wastepaper bins. 